In [2]:
import os
from dotenv import load_dotenv
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from geoalchemy2 import Geometry
from sqlalchemy import create_engine
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import SQLDatabaseChain


In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [5]:
llm=ChatOpenAI(model_name="gpt-4")

In [6]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)

db = SQLDatabase(engine=engine)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [7]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)

db = SQLDatabase(engine=engine)
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [8]:
db_chain.run("how many tables are there?")



> Entering new SQLDatabaseChain chain...
how many tables are there? 
SQLQuery:SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public';
SQLResult: [(11,)]
Answer:There are 11 tables.
> Finished chain.


'There are 11 tables.'

In [9]:
agent_executor.run("How many tables are there in the database?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: treestratum, operation, datasource, specialfeature, treestandsummary, restriction, stand, assortment, specification, stand_4326, treestand
Thought:I now know the number of tables in the database.
Final Answer: There are 11 tables in the database.

> Finished chain.


'There are 11 tables in the database.'

In [11]:
agent_executor.run("What's the total sum of areas in the stand_4326 table?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
Observation: treestratum, operation, datasource, specialfeature, treestandsummary, restriction, stand, assortment, specification, stand_4326, treestand
Thought:I see the stand_4326 table is available. I should now check the schema to know the relevant columns for area.
Action: schema_sql_db
Action Input: stand_4326
Observation: 
CREATE TABLE stand_4326 (
	id INTEGER, 
	standid BIGINT, 
	parcelid BIGINT, 
	realestateid BIGINT, 
	standnumber BIGINT, 
	standnumberextension VARCHAR(2), 
	maingroup BIGINT, 
	subgroup BIGINT, 
	fertilityclass BIGINT, 
	soiltype BIGINT, 
	drainagestate BIGINT, 
	ditchingyear BIGINT, 
	thinningyear BIGINT, 
	developmentclass VARCHAR(2), 
	standquality BIGINT, 
	maintreespecies BIGINT, 
	accessibility BIGINT, 
	standinfo VARCHAR, 
	datasource BIGINT, 
	growthplacedatasource BIGINT, 
	area DOUBLE PRECISION, 
	areadecrease DOUBLE PRECISION, 
	creationtime TIMESTAMP WITH TIME ZONE, 


'The total sum of areas in the stand_4326 table is 600761.402.'